In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew, LLM
# from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
import pdfplumber
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from utils import get_openai_api_key, get_serper_api_key
import os

In [3]:
llm = LLM(model="ollama/deepseek-r1:1.5b", api_base="http://localhost:11434")

In [29]:
# # Configure LTMSQLiteStorage for long-term memory
# ltm_storage = LTMSQLiteStorage(
#     db_path="./TailorMade AI/long_term_memory_storage.db" 
# )

In [30]:
from crewai_tools import (
    FileReadTool,
    ScrapeWebsiteTool,
    PDFSearchTool,
    SerperDevTool
)

# iiiii

In [6]:
pdf_path = "./Ahmad_Liaqat__Resume__28_03_2025.pdf"

In [7]:
# Extract text using pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    text = "\n".join([page.extract_text() or "" for page in pdf.pages])

# Save extracted text to a temporary file
temp_txt_path = "Ahmad_Liaqat__Resume__28_03_2025.txt"
with open(temp_txt_path, "w", encoding="utf-16") as f:
    f.write(text)

In [31]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=temp_txt_path)
semantic_search_resume = PDFSearchTool(
    pdf='./Ahmad_Liaqat__Resume__28_03_2025.pdf',
    config=dict(
        llm=dict(
            provider="ollama",
            config=dict(
                model="deepseek-r1:1.5b",
                base_url="http://localhost:11434",
            ),
        ),
        embedder=dict(
            provider="ollama",
            config=dict(
                model="bge-large",  # Use BGE for embeddings
                base_url="http://localhost:11434",
            ),
        ),
    ),
)

In [9]:
# from IPython.display import display, IFrame

# pdf_path = "Ahmad_Liaqat__Resume__28_03_2025.pdf"
# display(IFrame(pdf_path, width=400, height=600))

In [10]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on job posting to "
         "help job applicants",
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    tools=[scrape_tool, search_tool],
    verbose=True,
    llm=llm
)

In [11]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    llm=llm
)

In [12]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    backstory=(
       "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements." 
    ),
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    llm=llm
)

In [13]:
# Agent 4: Cover Letter Strategist
cover_letter_strategist = Agent(
    role="Cover Letter Specialist for Engineers",
    goal="Craft compelling and tailored cover letters that effectively showcase skills, experience, and motivation.",
    backstory=(
        "An expert in persuasive writing, you specialize in creating impactful cover letters that capture the "
        "attention of hiring managers. With a deep understanding of industry expectations, you ensure each letter "
        "aligns with the job's requirements while reflecting the applicant’s unique strengths and aspirations."
    ),
    tools=[scrape_tool, search_tool,
           read_resume, semantic_search_resume],
    verbose=True,
    llm=llm
)


In [14]:
# Agent 5: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm
)

In [15]:
# Task 1: task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [16]:
# Task 2: task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, personal portfolio ({portfolio_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [17]:
# Task 3: Task for Resume Strategist Agent - Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from previous tasks, "
        "analyze the job posting and identify necessary resume modifications. "
        "Generate a structured output specifying: \n"
        "- Bullet points to replace with improved wording in specific experience sections. \n"
        "- New bullet points to add under the relevant job experience. \n"
        "- Sections to remove or condense if irrelevant. \n"
        "- Skills or keywords that should be incorporated into the summary, skills, or experience sections. \n\n"
        "Ensure that all changes enhance job relevance while maintaining accuracy—do not fabricate details."
    ),
    expected_output=(
        "1. A structured list of resume modifications in the format:\n\n"
        "- **Experience: [Job Title, Company]**\n"
        "  - **Replace:** [Original bullet point] → [Updated bullet point].\n"
        "  - **Add:** [New bullet point].\n"
        "  - **Remove:** [Irrelevant bullet point].\n"
        "  - **Reword:** [Existing phrase] → [Revised phrase with job-related keywords].\n\n"
        "2. A list of general resume modifications (summary, skills, education).\n"
        "3. An updated resume reflecting the identified changes."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [18]:
# Task 4: Task for Cover Letter Strategist - Generate a Tailored Cover Letter
cover_letter_strategy_task = Task(
    description=(
        "Using the candidate's profile, resume and job requirements obtained from previous tasks, "
        "generate a brand-new, highly relevant, and compelling cover letter from scratch. \n\n"
        "Ensure the cover letter:\n"
        "- Has a strong, engaging opening that states the intent and enthusiasm for the role.\n"
        "- Clearly highlights the most relevant experiences and skills that align with the job.\n"
        "- Mention the individual location ({location}) and additional information ({additional_info}) as well."
        "- Uses industry-relevant keywords naturally for ATS optimization.\n"
        "- Demonstrates a direct connection between the candidate’s expertise and the job needs.\n"
        "- Includes a professional yet personalized closing statement with a call to action.\n"
        "- Strikes the perfect balance between quality and length (concise but impactful)."
    ),
    expected_output=(
        "A fully written cover letter tailored to the specific job posting, ensuring it:\n"
        "- Reads naturally and professionally.\n"
        "- Showcases the candidate’s strengths effectively.\n"
        "- Aligns perfectly with the job description without being overly generic or robotic.\n"
        "- Is formatted properly with a professional structure."
        "- Make sure the Cover letter is no longer then 2 paragraphs, make it short and bold the key keywords."
    ),
    output_file="tailored_cover_letter.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=cover_letter_strategist
)

In [19]:
# Task 5: task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task, cover_letter_strategy_task],
    agent=interview_preparer
)

In [20]:
job_application_crew = Crew(
    agents=[researcher, 
            profiler,
            resume_strategist,
            cover_letter_strategist,
            interview_preparer],
    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           cover_letter_strategy_task,
           interview_preparation_task],
#     memory = True
#     # Long-term memory with LTMSQLiteStorage
#     long_term_memory=LongTermMemory(storage=ltm_storage),

#     # Short-term memory with LTMSQLiteStorage (for context-based recall)
#     short_term_memory=ShortTermMemory(storage=ltm_storage),

#     # Entity memory with LTMSQLiteStorage (to track important entities)
#     entity_memory=EntityMemory(storage=ltm_storage),
    
    verbose=True
)

In [21]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/view/4114087574',
    'github_url': 'https://github.com/ahmad786writes',
    'portfolio_url': 'https://ahmad786writes.netlify.app/',
    'location': 'Riyadh, Saudi Arabia',
    'additional_info': 'Transferable Iqama',
    'personal_writeup': """AI Engineer based in Riyadh, Saudi Arabia, with years of experience in NLP, LLMs, and Computer Vision, specialized in GPT, BERT, and
                        different Transformer‑based architectures. Proven expertise in fine‑tuning and deploying state‑of‑the‑art AI models for real‑world
                        applications, including text generation, language understanding, and AI‑driven automation. Skilled in SQL, AWS, GCP, TensorFlow,
                        PyTorch, and Hugging Face, with a track record of integrating AI models into scalable cloud‑based environments."""
}

In [22]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e3f19a8f-3907-4cd2-9b66-c51829d10af5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0f519a0b-1ea6-4dfb-9ab4-72fa2312ad54
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 0f519a0b-1ea6-4dfb-9ab4-72fa2312ad54
│      Status: Executing Task...
└── 📋 Task: e30b6b06-3aef-4090-888b-07e28d474605
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 0f519a0b-1ea6-4dfb-9ab4-72fa2312ad54
│      Status: Executing Task...
└── 📋 Task: e30b6b06-3aef-4090-888b-07e28d474605
       Status: Executing Task...
    └── 🤖 Agent: Tech Job Researcher
            Status: In Progress

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://www.linkedin.com/jobs/view/4114087574) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


...



# Agent: Engineering Interview Preparer
## Final Answer: 
my best complete final answer to the task.




🚀 Crew: crew
├── 📋 Task: 0f519a0b-1ea6-4dfb-9ab4-72fa2312ad54
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
├── 📋 Task: e30b6b06-3aef-4090-888b-07e28d474605
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: In Progress
│   │   └── 🧠 Thinking...
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
├── 📋 Task: fac7eb58-0e7f-4972-91a5-0242a897efea
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
├── 📋 Task: ba82da77-3da8-4f8a-9a91-779c0fffd89d
│      Assigned to: Cover Letter Specialist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Cover Letter Specialist for Engineers
│           Status: ✅ Completed
└── 📋 Task: 61ee03a0-d01a-437f-9651-a7beb1a03ec0
       Status: Executing Task...
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 0f519a0b-1ea6-4dfb-9ab4-72fa2312ad54
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
├── 📋 Task: e30b6b06-3aef-4090-888b-07e28d474605
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: In Progress
│   │   └── 🧠 Thinking...
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
├── 📋 Task: fac7eb58-0e7f-4972-91a5-0242a897efea
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
├── 📋 Task: ba82da77-3da8-4f8a-9a91-779c0fffd89d
│      Assigned to: Cover Letter Specialist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Cover Letter Specialist for Engineers
│           Status: ✅ Completed
└── 📋 Task: 61ee03a0-d01a-437f-9651-a7beb1a03ec0
       Assigned to: Engineering Interview Preparer
       Status: ✅ Completed
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 61ee03a0-d01a-437f-9651-a7beb1a03ec0                                                                     │
│  Agent: Engineering Interview Preparer                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e3f19a8f-3907-4cd2-9b66-c51829d10af5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [26]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

**Experience:** [Job Title, Company]  
- **Replace:** "Developed custom RNNs for stock price prediction and sentiment analysis." → "Developed end-to-end solutions using advanced neural networks for stock price prediction, sentiment analysis, and customer feedback optimization."  
- **Add:** "Collaborated with FAIR-AI to deploy GPT models in cloud environments for real-world applications." → "Collaborated with FAIR-AI to deploy GPT models optimized for production use cases across diverse industries."  
- **Remove:** [Irrelevant bullet point].  
- **Reword:** "Developed custom RNNs for stock price prediction and sentiment analysis." → "Designed and implemented end-to-end solutions for predicting stock trends, analyzing customer feedback, and optimizing decision-making processes using advanced neural networks."

---

### General Resume Modifications (Summary)  
- Highlight the integration of data science techniques with web development expertise.  
- Emphasize a focus on bridging technical skills with real-world applications relevant to the role.

---

### General Resume Modifications (Skills)  
- **Programming Languages:** Python, Java, C++, JavaScript, Rust, Data Science tools like Hugging Face, TensorFlow, PyTorch.  
- **Model Integration & Deployment:** Deployed GPT models optimized for production use cases across diverse industries.  

---

### General Resume Modifications (Contributions)  
- Collaborated with FAIR-AI to deploy GPT models in cloud environments for real-world applications.  
- Enhanced large language models using quantization and efficient training techniques.  

---

### General Resume Modifications (Personal Interests)  
- Focused on making AI systems more transparent, accountable, and ethical.

In [27]:
display(Markdown("./interview_materials.md"))

my best complete final answer to the task.

In [28]:
display(Markdown("./tailored_cover_letter.md"))

**  
Certainly! Here's an organized and professional cover letter tailored to your request:  

---

### Cover Letter  

[Your Name]  
[Your Address]  
[City, State, ZIP Code]  
[Email Address]  
[Phone Number]  
[Date]  

**-engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering engineering